- LOAD BASE FILES
- CONVERT TO 2Hz
- FIND BEST NETWORK

In [6]:
from math import floor, ceil
import numpy as np
import pandas as pd
from sklearn.metrics import *

In [2]:
columns = {'arch':0, 'normal_1_acc':2, 'normal_2_acc':4, 'f1_acc':6, 'f2_acc':8, 'f3_acc':10, 'f4_acc':12}

numbered_columns = ['normal_1_acc', 'normal_2_acc', 'f1_acc', 'f2_acc', 'f3_acc', 'f4_acc']

In [4]:
'''
NOTE THAT THE "NORMAL 2" PATTERN COULD BE OUT OF PATTERN. REQUIRED 2 LINES as 2x2 array. IF THIS HAPPENED, MUST BE MANUALLY CONVERTED TO THE REQUIRED PATTERN AS BELOW!

REQUIRED
[[1. 0.]
 [0. 0.]]

OUT OF PATTERN
[[1.]]
'''

with open('./outputs/output_baseline_dr_2-ts_0.8-lf_1-limit_5000-wl_1.txt') as f:
    lines = f.readlines()

height = 13
times = floor(len(lines)/height)

output = pd.DataFrame(np.zeros((times, len(columns))), columns=columns)

for k, column_name in enumerate(columns):
    column_init_position = columns[column_name]

    for time in np.arange(0, times):
        line_number = column_init_position+(time*height)
        value = lines[line_number].rstrip("\n")

        if (column_name in numbered_columns):
            value = float(value)

        output.iloc[time,k] = value

f.close()

In [11]:
q = output.copy()
samples = [36000, 5000, 5000, 5000, 5000, 5000]

q['samples_n1'] = q['normal_1_acc'] * samples[0] / 100
q['samples_n2'] = q['normal_2_acc'] * samples[1] / 100
q['samples_f1'] = q['f1_acc'] * samples[2] / 100
q['samples_f2'] = q['f2_acc'] * samples[3] / 100
q['samples_f3'] = q['f3_acc'] * samples[4] / 100
q['samples_f4'] = q['f4_acc'] * samples[5] / 100

idx = []
for i, e in q.iterrows():
    idx.append(i)

q1 = q.loc[idx]

for n, e in q1.iterrows():
    tp = ceil(e['samples_n1']) + ceil(e['samples_n2'])
    fp = samples[0] + samples[1] - tp
    tn = ceil(e['samples_f1']) + ceil(e['samples_f2']) + ceil(e['samples_f3']) + ceil(e['samples_f4'])
    fn = samples[2] + samples[3] + samples[4] + samples[5] - tn

    y_true = [0] * (samples[0]+samples[1]) + [1] * (samples[2]+samples[3]+samples[4]+samples[5])
    y_pred = [0] * tp + [1] * fp + [0] * fn + [1] * tn

    q1.loc[n, 'ba'] = balanced_accuracy_score(y_true, y_pred)
    q1.loc[n, 'precision'] = precision_score(y_true, y_pred)
    q1.loc[n, 'recall'] = recall_score(y_true, y_pred) #(tn / (tn+fn))
    q1.loc[n, 'specificity'] = tn / (tn+fp)

In [8]:
q1.iloc[:, :].to_pickle('outputs/lof_resultado.pkl')